In [1]:
import os
import json
from difflib import SequenceMatcher
def search_relevance (score):
        match,relevance = "no",""
        if score>=99:
            match,relevance = "full",""
        elif score >=90:
            match,relevance = "partial","high"
        elif score >=70:
            match,relevance = "partial","medium"
        elif 0<score <70:
            match,relevance = "partial","low"
        return match,relevance

In [2]:
import json
import re
import fitz

from pdf2txt import convert_pdf
from bs4 import BeautifulSoup,NavigableString
from difflib import SequenceMatcher
import os


from nltk.corpus import stopwords

In [3]:
def getMatchTypeRelevance(a) : 
#     print(a)
    matchtype_agg=""
    relevance_agg=""
    for i in a:
        matchtype = i.get("matchType","")
        relevance = i.get("relevance","")
        
        
        if matchtype=="partial" :
            
            if matchtype_agg not in("no"):
                matchtype_agg=matchtype
            elif matchtype_agg=="":
                matchtype_agg=matchtype
                
            
                
        elif matchtype=="full" :
            if matchtype_agg not in("partial","no"):
                matchtype_agg=matchtype
            elif matchtype_agg=="":
                matchtype_agg=matchtype
        elif matchtype=="no":
            matchtype_agg=matchtype
        
        if relevance=="low" :
            relevance_agg=relevance
        elif relevance=="medium" :
            
            if relevance_agg not in("low"):
                relevance_agg=relevance
            elif relevance_agg=="":
                relevance_agg=relevance
            
        elif relevance=="high" :
            
            if relevance_agg not in("medium","low"):
                relevance_agg=relevance
            elif relevance_agg=="":
                relevance_agg=relevance
            
    if matchtype_agg=="no" or matchtype_agg=="full":
        relevance_agg=""
   
    return matchtype_agg,relevance_agg

In [4]:
from tabula import read_pdf

In [5]:
import fitz

In [6]:
def get_html(doc_name,pages,doc_path=r"PDBI"):
    paragraph=""
    doc=fitz.open(os.path.join(doc_path,doc_name))
    for page_no in pages:

        page = doc.loadPage(page_no-1)
        paragraph=paragraph+page.getText("html")+"\n"
    return paragraph

In [44]:
def generate_clean_html(doc_name,pages,doc_path=r"PDBI"):
    fitz_doc = fitz.open(os.path.join(doc_path,doc_name))
    html_full=''
    wording_ref=[]
    footer_dict={}
    total_pages=len(fitz_doc)
#     print(total_pages)
    for page_no in pages:
        page = fitz_doc.loadPage(page_no-1)
        

        page_html=page.getText("html")
       
        page_html=page_html.replace('position','page:{};position'.format(page_no))
        soup = BeautifulSoup(page_html, 'html5lib')
        toc_page_no=[]
        dot_count=0
        introflag=False

        for p in soup.findAll('p'):        
            p_str=str(p)
            dot_count+=p_str.count('.')

            try:        
                xcord=int(x_cordinate_Re.findall(p_str)[0])
            except:
                xcord=0 
            try:
                ycord=int(y_cordinate_Re.findall(p_str)[0])
            except:
                ycord=0

            for span in p.findAll('span'):
                text=span.text.strip()
                try:
                    toc_page=int(span.text)


                    if page_no< total_pages and xcord>500:
                        toc_page_no.append(toc_page)                   
                except Exception as err:
                    pass
                try:
                    font_size=float(font_size_RE.search(p_str).group(1))
                except Exception as err:
                    print(err)
                    font_size=8

                if xcord < 150 and ycord >720 and font_size <10 :


                    wording_ref.append(text)
                if  ycord >720  and font_size <10 :
                    footer_dict[text]=ycord

 

 

 


        if len(toc_page_no) >10 or dot_count>1500 :
#             print('table of content',page_no)
            continue
        if 'img' in page_html:
#             print('Image',page_no)
            continue
        if 'Introduction' in page_html and 'www.chubb.com' in page_html:
#             print('Intro',page_no)
            continue

#         if page_no <=16:
#             df=read_pdf(file,  pages=page_no)
#             if df:
#                 print('TABLE FOUND IN ',page_no)
#                 continue


        html_full+=page_html

    wording_ref=[i for i in wording_ref if len(i)>2]
    wording_reference_p_text=max(set(wording_ref), key = wording_ref.count)
    footer_y_cord=footer_dict[wording_reference_p_text]-20
    wording_reference=[i for i in wording_reference_p_text.split('    ') if len(i)>0][0]
    
    html_full=html_full.replace('width:-2pt;height:-2pt','width:596pt;height:842pt')

 

 

 

    soup = BeautifulSoup(html_full, 'html5lib')

    for p in soup.findAll('p'):
        p_str=str(p)
        text=p.text.strip()


        try:        
            xcord=int(x_cordinate_Re.findall(p_str)[0])
        except:
            xcord=0 
        try:
            ycord=int(y_cordinate_Re.findall(p_str)[0])
        except:
            ycord=0

        try:
            font_size=float(font_size_RE.search(p_str).group(1))
        except Exception as err:
            print(err)
            font_size=8
        if "policy number:" in p.text.lower() and ycord <100:
#             print(p.text)
            p.string=''
        if  font_size >40 :
#             print(p.text,'WATERMARKKK')
            p.string=''
    
        if len(p.text)>4:    
            if ycord >footer_y_cord and (font_size <8 or (p.text.strip()==wording_reference.strip())):
#                 print(p.text,'refff')
                p.string=''
        if font_size < 7 :
#             print(p.text,'less than 7 ')
            p.string=''
    

 

 

    return soup,wording_reference

In [45]:
# html_content = str(soup1)
# Html_file= open("doc_mod.html","w",encoding='utf-8')
# Html_file.write(html_content)
# Html_file.close()


In [46]:

pointsRE_heading=re.compile('(?:\s*\([a-z]{1,3}\)|[A-Z]{1}\s+[a-zA-Z0-9_\s]{5})')
stop_words = list(set(stopwords.words('english')))
y_cordinate_Re=re.compile(';top:(\d{1,5})pt')  #top:766pt
x_cordinate_Re=re.compile(';left:(\d{1,5})pt')
height_re=re.compile('height:(\d{1,5})pt')
font_size_RE = re.compile("font-size:(.*?)pt")
 


In [47]:
import diff_match_patch as dmp_module

dmp = dmp_module.diff_match_patch()


def get_dif(str1,str2):
    dmp.Diff_Timeout = 0
    
#     if 10000 <len(str2)<20000:
#          dmp.Diff_Timeout = 20
#     elif 20000<=len(str2)<25000:
#         dmp.Diff_Timeout = 60
#     elif 25000<=len(str2)<30000:
#         dmp.Diff_Timeout = 120
#     elif 30000<=len(str2)<35000:
#         dmp.Diff_Timeout = 180
#     elif len(str2)>35000:
#         dmp.Diff_Timeout = 240
    

#     print(        dmp.Diff_Timeout )   
    d = dmp.diff_main(str1, str2)
    dmp.diff_cleanupSemantic(d)

    paragraph=dmp.diff_prettyHtml( d)
    return paragraph,d

In [48]:
def sub_match(str1,str2):

    if str1 in str2 or str2 in str1:
        return True
    return False

In [49]:
def long_substring(string1,string2):
    
    if string1 in string2:
#         print(string1)
        return True,string1
    sequence_matcher = SequenceMatcher(
        None, string1, string2)
    match = sequence_matcher.find_longest_match(
        0, len(string1), 0, len(string2))
    answer=string1[match.a:match.a+match.size]
    if len(answer.strip())<4:
        return False,""
#     print(answer)
    return True ,answer

In [50]:
def get_html_with_diff(doc_name,pages,key_values,dist_,start):
    soup,wording_reference=generate_clean_html(doc_name,pages)
    soup_p_list=[]
#     for p in soup.find_all('p'):
#         text =""
#         for span in p.find_all('span'):
#             text=text+span.text 
#         soup_p_list.append(p.text+)
#     for divs in soup.findAll('p'):
#         text=""
#         for span in divs.find_all('span'):
#             text=text+''+span.text
#         soup_p_list.append(text)

#     print("".join(i for i in soup_p_list))
    soup= get_highlights(key_values,soup,start)
    
#     p_index=0
#     for match in soup.findAll('p'):
#         if p_index in highlight_index.keys():
#             hight_value=highlight_index.get(p_index,"")

#             for span in match.findAll('span'):
#                 if len(span.string.strip())>1:

#                     span.string="<ext>{}</ext>".format(span.string)
#         p_index+=1
    total=0
    for i,item in enumerate(dist_):
        
        p_index=-total
#         print(p_index)
        
        tag,s_text=item[1].strip().split("$")
        total=total+13+len(item[0])
#         print(total,13+len(item[0]))
        checkFalg=False 
        total_text=""
        for p in soup.findAll('p'):
            text=""
            
       
            for span in p.find_all('span'):
                text+=span.text
            total_text+=text
            dist_count=text.count("<dist>")
            text=text.replace("<ext>","").replace("</ext>","")
          
            if p_index<=int(tag)<=p_index+len(text) and len(text.strip())>1:
                new_div = soup.new_tag('span')
                new_div["style"]=p.span["style"]
                
                new_div.string="<dist>{}</dist>".format(item[0])
                p.insert(dist_count, new_div)
                
                break
                
            p_index+=len(text)
#         print("***************************")
                
                
    paragraph=str(soup)
#     paragraph = paragraph.replace("&lt;ext&gt;","<mark>")
#     paragraph = paragraph.replace("&lt;/ext&gt;","</mark>")
    return paragraph

In [51]:
def get_differences(str1,str2,category):
#     str1=str1.replace(" MN MasterPackage 2019",'')
#     str2=str2.replace(" MN MasterPackage 2019",'')

    diff_,get_=get_dif(str1,str2)
#     print(diff_)
    f=open(category+"pretty.html","w",encoding="utf-8")
    f.write(diff_)
#     print(get_)
    removedminus=[ i for i in get_ if i[0]!=-1]
#     print("".join(i[1]for i in removedminus))
    
    start=""
    
        
    key_values={}
    total =0
    for index,a in enumerate(removedminus):
        item,value=a

        if item==1:
            
#             extra_spaces=len(value) - len(value.lstrip(' '))
#             if extra_spaces>1:
#                 total1=extra_spaces+total-1
#             else:
#                 total1=total
            key_values[total]=value
       
        total+=len(value)
    
    
    pre_text=""
    pre_tag=0
    dist_=[]
    search_highlight={}
    flag=True
    total=0
    total_text=""
    
    for item in get_:
        
        tag,text=item
        total_text+=text
        if len(text.strip())>3 and tag==0 and flag :
            flag=False
            start = text.split()[0]

            if start.lower() in stop_words:
                start=text[:50]
        
        if tag==0:
            prev_index=tag
            prev_value= text
        if pre_tag==-1:
            extra_spaces=len(value) - len(value.lstrip(' '))
            
            total1=extra_spaces+total           
            dist_.append((pre_text,str(total1)+"$"+text))

            
        pre_tag=tag
        pre_text=text

        if tag==0 or tag==1:
            total+=len(text)
        
    dist_=[i for i in dist_ if len(i[0].strip())>3 and  len(i[1].strip())>3]

    return key_values,dist_,start
    

In [52]:
def get_highlights(key_values,soup,start) :
    i= start
#     print(i)
    highlight_index={}
    total=0
    for index1,key in key_values.items():
        content =""
        if len(key)>3:

            total_len=0

            flag=False
            
            for  p in  soup.find_all('p'):
                a=""
                for span in p.find_all('span'):
                    a=a+span.text 
               
                checkFlag,answer=    long_substring(key,a)

               
                if len(a.strip())>6 and checkFlag :
                    try:


                        p_text_len=total_len+a.index(key)
                        
                        
                        if index1==p_text_len:

                            for span in p.findAll("span"):
                                span.string="<ext>{}</ext>".format(span.text)
                    

                           


                    except:
                       
                            
                            
                        try:

                            if total_len>=(index1+key.index(a)-40) :
                                
                                for span in p.findAll('span'):
                                    
                                    span.string="<ext>{}</ext>".format(span.text)
                                    

                                

  
                        except:

                            
                            if total_len+a.index(answer)==index1+key.index(answer):
                                for span in p.findAll('span'):
                                    
                                    span.string="<ext>{}</ext>".format(span.text)

                                 
                                

                if a.strip() and  a.startswith(i) and not flag :

                    flag=True
                if flag :
                    
                    total_len+=len(a.replace("<ext>","").replace("</ext>",""))
                    
                    content+=a

#         print("**************************")
    return soup

In [53]:
# %%time
# import os
# word_exclusion=word_content[0]["exclusion"]
# doc_exclusion=doc_content[0]["exclusion"]

 

In [54]:



# from azure.cosmos import exceptions, CosmosClient, PartitionKey

# cosmo_endpoint="https://nf-poc-cdb-sql.documents.azure.com"
# cosmo_key="iEcEfrxYe0Fm9QtoxDrOpLvGsfzjowwybULlWT9Uz4XxV4RmOIAnRuLdgRFUu1LPU5Vwk3UGivRrPrxnk7083w=="
# client = CosmosClient(cosmo_endpoint, cosmo_key)
# database=client.get_database_client('policy-analysis')
# tt_documents_container=database.get_container_client('trace_files_dev')
# trace_result_container=database.get_container_client('trace_results_dev')

# # In[12]:


# # wording_query="SELECT * FROM c WHERE c.file_type ='base wording' "
# wording_query="SELECT * FROM c WHERE c.title ='MasterPackage Multinational-UK-2019 (Limit of Loss S 1 & 2)-SPECIMEN-(to 31-12-2020)'"


# wordings = list(tt_documents_container.query_items(
#     query=wording_query,
#     enable_cross_partition_query=True
# ))


# # In[19]:


# # document_query="SELECT * FROM c WHERE c.file_type ='policy document'  "
# document_query="SELECT * FROM c WHERE c.title ='test_7_extra_paragraph_document'"



# documents = list(tt_documents_container.query_items(
#     query=document_query,
#     enable_cross_partition_query=True
# )) 





        



# result_query="SELECT * FROM c"



# results = list(trace_result_container.query_items(
#     query=result_query,
#     enable_cross_partition_query=True
# ))


In [55]:
# wording_content=wordings[0]['contents']
# document_content=documents[0]['contents']


# document_name=list(document_content.keys())[0]
# # print("document_name",document_name)
# wording_name=list(wording_content.keys())[0]
# doc_content=document_content[document_name]
# word_content=wording_content[wording_name]

In [56]:
# word_exclusion=word_content[0]["misc"]
# doc_exclusion=doc_content[0]["misc"]
# key="Section 0"

# f= open("final3"+".html","w",encoding="utf-8")
# pages=doc_exclusion[key][1]
# pages.sort()
# key_values,dist_,start=get_differences(word_exclusion[key][0],doc_exclusion[key][0],'exclusion')
# # print("start",start)
# # start="Condition#"
# # print("$$$$$$$$$$$$$$$$$$$$$$$$$$")
# # start="Exclusion"
# key_values={key:value for key,value in key_values.items() if len(value.strip())>3}
# dist_=[dist for dist in dist_ if len(dist[1].split('$')[1].strip()) >3 ]
# dist_
# soup= get_html_with_diff(document_name,pages,key_values,dist_,start)
# paragraph=str(soup)
# paragraph = paragraph.replace("&lt;ext&gt;","<mark>")
# paragraph = paragraph.replace("&lt;/ext&gt;","</mark>")
# dist_


# f.write(paragraph)

In [68]:



from azure.cosmos import exceptions, CosmosClient, PartitionKey

cosmo_endpoint="https://nf-poc-cdb-sql.documents.azure.com"
cosmo_key="iEcEfrxYe0Fm9QtoxDrOpLvGsfzjowwybULlWT9Uz4XxV4RmOIAnRuLdgRFUu1LPU5Vwk3UGivRrPrxnk7083w=="
client = CosmosClient(cosmo_endpoint, cosmo_key)
database=client.get_database_client('policy-analysis')
tt_documents_container=database.get_container_client('trace_files_prod')
trace_result_container=database.get_container_client('trace_results_prod')

# In[12]:


wording_query="SELECT * FROM c WHERE c.file_type ='base wording' "

wordings = list(tt_documents_container.query_items(
    query=wording_query,
    enable_cross_partition_query=True
))


# In[19]:


document_query="SELECT * FROM c WHERE c.file_type ='policy document'  "

documents = list(tt_documents_container.query_items(
    query=document_query,
    enable_cross_partition_query=True
))


# In[29]:
result_query="SELECT * FROM c"



results = list(trace_result_container.query_items(
    query=result_query,
    enable_cross_partition_query=True
))



        




    



In [69]:
results_id=[i['id'] for i in results]

In [70]:
len(results_id)

0

In [71]:
list(set(([i['refCode'] for i in wordings])))

['PDBI 04/2018 (SE)',
 'PDBI 05/2017',
 'PDBI 04/2018',
 'Multinational PDBI 05/2017',
 'UK/CCI/CIS/MP_16/16 – POL 2/06/2017 v8',
 'Multinational PDBI 04/2018 (SE)',
 'Multinational PDBI 04/2018',
 'MN MasterPackage 2019',
 'MasterPackage 01/09/2017']

In [72]:
list(set([i['refCode'] for i in documents]))

['UK/CCI/CIS/MP_16/16',
 'PDBI 04/2018 (SE)',
 '16/10/17',
 'Mul ti national PDBI 04/2018 (SE)',
 'MasterPackage 2019',
 'UK/CCI/CIS/MP_16/16 – POL 19/07/2017 v8',
 'Multinational PDBI 05/2017',
 'PDBI 04/2018',
 'Multinational PDBI 04/2018 (SE)',
 'MasterPackage 01/04/2018',
 'SCH 19/06/2018 v1',
 'MN MasterPackage 2019']

In [73]:
# documents=[i for i in documents if i['title'].startswith('test')]

In [74]:
k=0
for i in [i['refCode'] for i in documents]:
    for j in [i['refCode'] for i in wordings]:
        if i==j:
            print(j)
            k+=1
            print(k)
            

Multinational PDBI 05/2017
1
MN MasterPackage 2019
2
MN MasterPackage 2019
3
MN MasterPackage 2019
4
MN MasterPackage 2019
5
Multinational PDBI 05/2017
6
MN MasterPackage 2019
7
MN MasterPackage 2019
8
MN MasterPackage 2019
9
MN MasterPackage 2019
10
MN MasterPackage 2019
11
MN MasterPackage 2019
12
Multinational PDBI 04/2018 (SE)
13
PDBI 04/2018
14
MN MasterPackage 2019
15
MN MasterPackage 2019
16
Multinational PDBI 05/2017
17
Multinational PDBI 04/2018 (SE)
18
PDBI 04/2018 (SE)
19
Multinational PDBI 05/2017
20
Multinational PDBI 04/2018 (SE)
21
MN MasterPackage 2019
22
MN MasterPackage 2019
23
MN MasterPackage 2019
24
MN MasterPackage 2019
25
PDBI 04/2018 (SE)
26
Multinational PDBI 05/2017
27
Multinational PDBI 04/2018 (SE)
28
PDBI 04/2018 (SE)
29
MN MasterPackage 2019
30
MN MasterPackage 2019
31
Multinational PDBI 05/2017
32
Multinational PDBI 05/2017
33
Multinational PDBI 04/2018 (SE)
34
PDBI 04/2018
35
MN MasterPackage 2019
36
MN MasterPackage 2019
37
PDBI 04/2018
38
Multinational

In [75]:
[i['title'] for i in wordings]

['Chubb PDBI - UK - (from 01-05-2017 until 31-03-2018) - Specimen',
 'Chubb Multinational PDBI - UK - (from 01-04-2018 to 31-12-18) - SPECIMEN',
 'Chubb PDBI - UK - (from 01-04-18 to 31-12-18) - SPECIMEN',
 'MasterPackage Multinational-UK-2019 (Limit of Loss S 1 & 2)-SPECIMEN-(to 31-12-2020)',
 'Chubb Multinational PDBI - UK - 04-2018 (SE - from 01-01-2019) - SPECIMEN',
 'Chubb PDBI - UK - 04-2018 (SE - from 01-01-2019 to 31-12-20) - SPECIMEN',
 'MasterPackage Multinational-UK-2019 (Full Value S 1 & 2)-SPECIMEN (to 31-12-2020)',
 'Chubb Multinational PDBI - UK - (from 01-05-2017 until 31-03-2018) - Specimen',
 'MasterPackage -MP_16 _ 02-06-2017- No end date available',
 'MasterPackage policy wording 01_09_2017 (no end date available)']

In [76]:
k=0
for wording in wordings:
    
    refCode_wording=wording["refCode"]

    for document in documents:
        refCode_document=document["refCode"]
        wording_start_date=wording['effectiveDate'] 
        wording_end_date=wording['expiryDate'] 
        document_start_date=document['effectiveDate']
        if refCode_document==refCode_wording and  wording_start_date<=document_start_date<=wording_end_date:
            k+=1
            print(k)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155


In [40]:
len(documents)

196

In [77]:
document_set=documents[:50]

In [78]:


result_query="SELECT * FROM c"

results = list(trace_result_container.query_items(
    query=result_query,
    enable_cross_partition_query=True
))

results_id=[i['id'] for i in results]

final_list=[]
for wording in wordings:
    
    refCode_wording=wording["refCode"]

    for document in document_set:
        refCode_document=document["refCode"]
        wording_start_date=wording['effectiveDate'] 
        wording_end_date=wording['expiryDate'] 
        document_start_date=document['effectiveDate']
        try:
            if refCode_document==refCode_wording and  wording_start_date<=document_start_date<=wording_end_date:            
                wording_content=wording['contents']
                document_content=document['contents']
                


                document_name=list(document_content.keys())[0]

                wording_name=list(wording_content.keys())[0]
                id_name=wording_name.replace(".pdf","")+document_name.replace(".pdf","")
                print(id_name)
                
                
                if id_name in results_id:
                    print('Already Exist........')
                    continue

                final_list.append([wording_name,document_name,refCode_document,refCode_wording,wording["effectiveDate"],document["effectiveDate"]])

                doc_content=document_content[document_name]
                word_content=wording_content[wording_name]

                print(document_name)
                matches={}

                total_category_MR=[]
                word_content[0].pop("pages", None)
                doc_content[0].pop("pages", None)

                for category in word_content[0].keys():

                    category_content_wording=word_content[0][category]
                    category_content_document=doc_content[0][category]
                    results=[]
                    total_section_MR=[]
                    total_section_score=0
                    total_sections=0
    #                 print(category_content_document.keys())




                    for section_w in category_content_wording.keys():
                        new_dict ={}
                        keyFlag=False
                        ##keys are present in both the wording and document

                        if section_w not in category_content_document.keys():
                            for section_d in category_content_document.keys():

                                if section_w in section_d or section_d in section_w:
                                    key_wording=section_w
                                    key_document=section_d
                                    keyFlag=True


                        else:
                            key_wording=section_w
                            key_document=section_w

                            keyFlag=True

                        if keyFlag:
    #                         print(key_wording,"*",key_document,category)
                            section_content,pages=category_content_document[key_document]

                            wording_section_value,wording_section_pages=category_content_wording[key_wording]
                            pages.sort()

                            wording_section_pages.sort()
                            ##when wording pages and document pages are there we will compare
                            if len(pages)>=1 and len(wording_section_pages)>=1:

                                key_values,dist_,start=get_differences(wording_section_value,section_content,category)
                                dist_=[dist for dist in dist_ if len(dist[1].split('$')[1].strip()) >3 ]
                                key_values={key:value for key,value in key_values.items() if len(value.strip())>3 and refCode_document!=value}

                                score=SequenceMatcher(None,wording_section_value,section_content).ratio()*100
                                if not key_values and not dist_:
                                    score=100
                                if "SPECIAL EXTENSIONS APPLICABLE" in  section_w :
                                    score=100
                                    

                                matchType,relevance=search_relevance(score)
                                total_section_score+=score
                                total_sections+=1
                                new_dict["section"]=key_document


                                if (key_values or  dist_) and score <99:


                                    content_html= get_html_with_diff(document_name,pages,key_values,dist_,start)


                                    new_dict["content"]=content_html

                                new_dict["relevance"] = relevance
                                new_dict["matchType"] = matchType
                                new_dict["score"]=score
                                new_dict["document_page"]=pages[0]


                                new_dict["wording_page"]=wording_section_pages[0]


                                results.append(new_dict)
                                total_section_MR.append({"relevance":new_dict["relevance"],"matchType":new_dict["matchType"]})

                            else:
                                ##if there are documnet pages
                                ##elif there are wording pages
                                if len(pages)>=1:
                                    new_dict["section"]=key_document
                                    new_dict["content"]="<dist>{}</dist>".format(get_html(document_name,pages))

                                    new_dict["document_page"]=pages[0]
                                    new_dict["relevance"] = ""
                                    new_dict["matchType"] = "no"
                                    new_dict["score"]=0

                                    results.append(new_dict)
                                    total_section_MR.append({"relevance":new_dict["relevance"],"matchType":new_dict["matchType"]})


                                elif len(wording_section_pages)>=1:
                                    new_dict["section"]=key_wording
                                    new_dict["content"]="<dist>{}</dist>".format(get_html(wording_name,wording_section_pages,doc_path=r"PDBI/wordings"))

                                    new_dict["wording_page"]=wording_section_pages[0]
                                    new_dict["relevance"] = ""
                                    new_dict["matchType"] = "no"
                                    new_dict["score"]=0

                                    results.append(new_dict)
                                    total_section_MR.append({"relevance":new_dict["relevance"],"matchType":new_dict["matchType"]})




                        ###if the key it not there in document we should show wording pages with "No match"
                        if not keyFlag:
                            
                            _,pages_word=category_content_wording[section_w]

                            if len(pages_word)>=1:
                                pages_word.sort()
                                new_dict["section"]=section_w


                                new_dict["content"]="<dist>{}</dist>".format(get_html(wording_name,pages_word,doc_path=r"PDBI/wordings"))
                                new_dict["relevance"] = ""
                                new_dict["matchType"] = "no"
                                new_dict["score"]=0
                                new_dict["wording_page"]=pages_word[0]
                                results.append(new_dict)
                                total_section_MR.append({"relevance":new_dict["relevance"],
                                                                 "matchType":new_dict["matchType"]})
                    total_section_matchType,total_section_relevance=getMatchTypeRelevance(total_section_MR)
                    matches[category]= {"results":results,
                                        "matchType": total_section_matchType,"relevance":total_section_relevance}
                    total_category_MR.append({"relevance":total_section_relevance,
                                                         "matchType":total_section_matchType})  

                total_category_matchType,total_category_relevance=getMatchTypeRelevance(total_category_MR)
                
                insert_item={
                              "policyDetails": {
                                    "policyNo": document["policyNo"],
                                    "expiryDate": document[ "expiryDate"],
                                    "effectiveDate": document["effectiveDate"]
                                  },
                            "title": document_name.replace(".pdf",""),
                            "id":id_name,
                            "wordingTitle":wording_name.replace(".pdf",""),
                            "matchType": total_category_matchType,

                            "relevance":total_category_relevance,

                            "refCode": document["refCode"],

                             "country":"GB",
                            "lob":document["lob"],
                            "matches": matches
                            }
#                 f=open("bb"+document['title']+".json","w")

#                 f.write(json.dumps(insert_item))
                print("*************************************************")  
                try:
                    trace_result_container.create_item(body=insert_item)
                except Exception as e:
                    print("exception block",wording_name,document_name,e)
                    try:
                        trace_result_container.replace_item(item=document_name, body=insert_item)
                    except Exception as e1:
                        print("exception block1",e1,wording_name,document_name,e1)
        except Exception as err:
            print("higher level exception",err,wording_name,document_name)

Chubb PDBI - UK - (from 01-04-18 to 31-12-18) - SPECIMENREDACTED_2054BA20-4274-4906-8583-CB240343539E
REDACTED_2054BA20-4274-4906-8583-CB240343539E.pdf
*************************************************
MasterPackage Multinational-UK-2019 (Limit of Loss S 1 & 2)-SPECIMEN-(to 31-12-2020)test_1_section_name_document
test_1_section_name_document.pdf
*************************************************
MasterPackage Multinational-UK-2019 (Limit of Loss S 1 & 2)-SPECIMEN-(to 31-12-2020)REDACTED_9EA07FB4-511F-4DD3-8FFB-636C21AC5F0E
REDACTED_9EA07FB4-511F-4DD3-8FFB-636C21AC5F0E.pdf
*************************************************
exception block MasterPackage Multinational-UK-2019 (Limit of Loss S 1 & 2)-SPECIMEN-(to 31-12-2020).pdf REDACTED_9EA07FB4-511F-4DD3-8FFB-636C21AC5F0E.pdf (RequestEntityTooLarge) Message: {"Errors":["Request size is too large"]}
ActivityId: 9155ba3d-a4b2-4045-aebe-0c6c5dbf07e9, Request URI: /apps/80fd50c3-b734-4882-9357-6122a786ca0d/services/12c5648a-251e-4eea-ae61-b707

*************************************************
MasterPackage Multinational-UK-2019 (Full Value S 1 & 2)-SPECIMEN (to 31-12-2020)REDACTED_2D449370-E71A-44B4-B46B-5E68FCA8DC46
REDACTED_2D449370-E71A-44B4-B46B-5E68FCA8DC46.pdf
*************************************************
MasterPackage Multinational-UK-2019 (Full Value S 1 & 2)-SPECIMEN (to 31-12-2020)test_9_document
test_9_document.pdf
*************************************************
MasterPackage Multinational-UK-2019 (Full Value S 1 & 2)-SPECIMEN (to 31-12-2020)REDACTED_DBBD38FD-6F82-4D6D-899F-A2C14A180BF2
REDACTED_DBBD38FD-6F82-4D6D-899F-A2C14A180BF2.pdf
*************************************************
MasterPackage Multinational-UK-2019 (Full Value S 1 & 2)-SPECIMEN (to 31-12-2020)test_7_extra_paragraph_document
test_7_extra_paragraph_document.pdf
*************************************************
MasterPackage Multinational-UK-2019 (Full Value S 1 & 2)-SPECIMEN (to 31-12-2020)REDACTED_1F427748-D7C9-4864-80FC-698EB9C75A31
RED

In [40]:
# new_keys={}
# for key_w in word_exclusion.keys():
#     keyFlag=False
#     key_wording=key_w
#     if key_w not in doc_exclusion.keys():
#         for key_doc in doc_exclusion.keys():

#             if key_w in key_doc or key_doc in key_w:
#                 key_wording=key_w
#                 key_document=key_doc
#                 keyFlag=True
#                 print(key_wording,"*",key_document)
#     else:
#         key_wording=key_w
#         key_document=key_w
#         print(key_wording,"*",key_document)
#         keyFlag=True
        
#     if not keyFlag:
#         print(key_w)
        
    
    
                    
            
     

In [ ]:
word_exclusion[key]

In [ ]:
doc_exclusion[key]

In [59]:
document_name

'REDACTED_0BCBCCDF-0CC2-465E-A32A-41818E5894EC.pdf'

In [ ]:
key_values

In [ ]:
a={"Conditions":"abc"}
b={"special Conditions":"def"}

In [ ]:

from itertools import product
from fuzzywuzzy import fuzz


partial_ratio = fuzz.partial_ratio( "special Conditions"," Section 4 Conditions")
print(partial_ratio)

In [ ]:
#                         if total_sections!=0:
#                             total_section_score= round(total_section_score/total_sections,2)

#                         total_section_matchType="Full"
#                         if total_section_score<99.5:
#                             total_section_matchType="partial"